# HTTP Requests in Python (needs updates for Foundry)
This notebook runs the HTTP requests provided in the `test.http` file using Python.

In [ ]:
# Install required packages

!pip install requests azure.identity python-dotenv

In [ ]:
import requests
import json
import os
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AccessToken
from dotenv import load_dotenv

load_dotenv(verbose=True, override=True)

# Define variables
subscription_id = os.environ.get('subscriptionId', '00000000-0000-0000-0000-000000000000')
rg = os.environ.get('rg', 'dummy-rg')
capability_name = os.environ.get('capabilityName', 'dummy-caphost')
foundry_name = os.environ.get('foundryName', 'dummy-foundry')
project_name = os.environ.get('projectName', 'dummy-project')
base_url = f'https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{rg}/providers/Microsoft.CognitiveServices/accounts/'

# Get token using DefaultAzureCredential
credential = DefaultAzureCredential()
token = credential.get_token("https://management.azure.com/.default")

# Define headers
headers = {
    'Authorization': f'Bearer {token.token}',
    'Content-Type': 'application/json'
}

### Get Hub Capability Host

In [ ]:
url = f'{base_url}{foundry_name}/capabilityHosts/?api-version=2025-06-01'
response = requests.get(url, headers=headers)
response_json = response.json()
print(json.dumps(response_json, indent=4))
print(f"Foundry using subnet: {response_json['value'][0]['properties']['customerSubnet'] if 'value' in response_json and len(response_json['value']) > 0 else 'No subnet found'}")

### Create Hub Capability Host

In [ ]:
url = f'{base_url}{foundry_name}/capabilityHosts/{foundry_name}?api-version=2025-06-01'
payload = {
  "properties": {
    "capabilityHostKind": "Agents",
    "customerSubnet": '<subnet_resource_id>'  # Replace with actual subnet resource ID
  }
}
#print(json.dumps(payload, indent=4))
response = requests.put(url, headers=headers, data=json.dumps(payload))
print(json.dumps(response.json(), indent=4))

### Delete Hub Capability Host
Capability host needs to be deleted if it's in **Failed** state. Project capability needs to be deleted before hub capability.

In [ ]:
url = f'{base_url}{foundry_name}/capabilityHosts/{foundry_name}?api-version=2025-06-01'
# avoid incidentally deleting the capability host
delete = False
if delete:
    response = requests.delete(url, headers=headers)
    print(json.dumps(response.json(), indent=4))

### Create Project Capability Host

In [ ]:
url = f'{base_url}{project_name}/capabilityHosts/{capability_name}?api-version=2025-06-01'
payload = {
    "properties": {
        "capabilityHostKind": "Agents",
        "aiServicesConnections": ["<TODO - replace with actual connection name>"],
        "vectorStoreConnections": ["<TODO - replace with actual connection name>"],
        "storageConnections": ["<TODO - replace with actual connection name>"],
        "threadStorageConnections": ["<TODO - replace with actual connection name>"],
    }
}
response = requests.put(url, headers=headers, data=json.dumps(payload))
print(json.dumps(response.json(), indent=4))

### Get Project Capability Host

In [ ]:
url = f'{base_url}{foundry_name}/projects/{project_name}/capabilityHosts/{capability_name}?api-version=2025-06-01'
response = requests.get(url, headers=headers)
print(json.dumps(response.json(), indent=4))

### Delete Project Capability Host

In [ ]:
url = f'{base_url}{foundry_name}/projects/{project_name}/capabilityHosts/{capability_name}?api-version=2025-06-01'
# avoid incidentally deleting the capability host
delete = False
if delete:
    response = requests.delete(url, headers=headers)
    print(json.dumps(response.json(), indent=4))

### POST to Azure OpenAI Service

In [ ]:
ai_token = credential.get_token("https://ai.azure.com")
headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + ai_token.token,
}

# get deployments
url = f'https://{foundry_name}.services.ai.azure.com/api/projects/{project_name}/deployments?api-version=v1'
response = requests.get(url, headers=headers)

# grab the first deployment name
deployment_name = response.json().get('value', [{}])[0].get('name', None)
print(f"Deployment name: {deployment_name}")
# print(json.dumps(response.json(), indent=4))

url = f'https://{foundry_name}.services.ai.azure.com/models/chat/completions?api-version=2024-05-01-preview'

payload = {
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Does Azure OpenAI support customer managed keys?"
        },
        {
            "role": "assistant",
            "content": "Yes, customer managed keys are supported by Azure OpenAI."
        },
        {
            "role": "user",
            "content": "Do other Azure AI services support this too?"
        }
    ],
    "model": deployment_name,

}
response = requests.post(url, headers=headers, data=json.dumps(payload))
print(json.dumps(response.json(), indent=4))